In [1]:
! pip install emoji

In [2]:
import pandas as pd
import numpy as np
import re
import datetime
import math
import time
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls /content/drive/My\ Drive/Colab\ Notebooks/immigration_sentiment_analysis/TweetProcessor/*.py
#!cat '/content/drive/My Drive/Colab Notebooks/immigration_sentiment_analysis/TweetProcessor/TweetProcessor.py'
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/immigration_sentiment_analysis/TweetProcessor')
import TweetProcessor as tp
tweet_processor = tp.TweetProcessor()

'/content/drive/My Drive/Colab Notebooks/immigration_sentiment_analysis/TweetProcessor/TweetProcessor.py'


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
# Tag Removal
def remove_tags(tweet):  
  return re.sub('@\S*', '', tweet)

def remove_outliers(series):
  x = series
  UPPERBOUND, LOWERBOUND = np.percentile(x, [1,99])
  y = np.clip(x, UPPERBOUND, LOWERBOUND)
  return pd.Series(y)

In [5]:
file = "/content/drive/My Drive/Colab Notebooks/immigration_sentiment_analysis/data/tweet_immigration_db.csv"
raw_data_df = pd.read_csv(file)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,7,12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
raw_data_df = raw_data_df[['text', 'retweet_count']]
raw_data_df = raw_data_df.dropna()
i = raw_data_df[((raw_data_df.retweet_count == 'retweet_count'))].index
raw_data_df = raw_data_df.drop(i)
raw_data_df['retweet_count'] = raw_data_df['retweet_count'].apply(lambda x : int(x))
raw_data_df['retweet_count'] = remove_outliers(raw_data_df['retweet_count'])
data = raw_data_df.sample(n = 18000,  random_state=2)

In [7]:
data

,text,retweet_count
19185,@FLOTUS @RedCross “But the First Lady’s weapon...,0.0
1002,RT @Free_Media_Hub: The fragmented politics of...,4.0
23412,"RT @UR_Ninja: At one point last year, a Feuerk...",13.0
76268,So now I have a 3 week extension on my visa an...,0.0
81408,RT @vinaysreeni: A chance to correct my errors...,27.0
...,...,...
22670,"@brainpicker @rgay At @LIRSorg, teleworking st...",0.0
42555,"RT @someshjha7: Thread: Today, the Delhi Polic...",14.0
19088,How #coronavirus hits #migrants and #asylum se...,0.0
43276,RT @UMD_GLAO: TWO IMMIGRATION APPOINTMENTS STI...,2.0


In [8]:
data = data.dropna()
ps = PorterStemmer()
clean_text = []
for tweet in data['text']:
    tweet = remove_tags(tweet)
    tweet_list = tweet_processor.tweet_pipeline(tweet)
    stemmed_tweet_list = []
    for w in tweet_list: 
      stemmed_tweet_list.append(ps.stem(w))
    clean_text.append(' '.join(stemmed_tweet_list))
    

In [9]:
data=data.reset_index()
data=data.drop(['index'], axis=1)
vectorizer = TfidfVectorizer()
clean_text_tfidf = vectorizer.fit_transform(clean_text)
clean_text_tfidf.shape

(18000, 16057)

In [10]:
data['text_tfidf'] = clean_text_tfidf.toarray().tolist()

In [ ]:
from math import sqrt
from sklearn import datasets, linear_model
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

data_X = data['text_tfidf'].tolist()
data_y = data['retweet_count']

X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.3, random_state=42)

regr = SGDRegressor()
regr.fit(X_train, y_train)

pred = regr.predict(X_test)

print('Mean squared error: %.2f' % sqrt(mean_squared_error(y_test, pred)))

print('Coefficient of determination: %.2f' % r2_score(y_test, pred))

Mean squared error: 43.97
Coefficient of determination: 0.01


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1187: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor

X = data['text_tfidf'].tolist()
y = data['retweet_count'].tolist()

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=42)

model = SGDRegressor(loss='squared_loss', penalty='l2', random_state=2, max_iter=5)
params = {'penalty':['none','l2','l1'],
          'alpha':[1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2, 0.1]}
gs = GridSearchCV(estimator=model,
                  param_grid=params,
                  scoring='neg_root_mean_squared_error',
                  n_jobs=1,
                  cv=5,
                  verbose=3)

gs.fit(X_train, y_train)

In [12]:
model = gs.best_estimator_
print(gs.best_params_)
print(gs.best_score_)

{'alpha': 0.0001, 'penalty': 'none'}
-41.28068345727


In [ ]:
y_pred = model.predict(X_test)